In [ ]:
!pip install -U -q "google-generativeai>=0.7.2"

In [2]:
import google.generativeai as genai
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
from google.colab import files
from PIL import Image
import io
import numpy as np
import time
import os  # Import the os module for file deletion

def process_image(image_path):
  """Processes a single image and returns OCR results as a JSON object."""
  # Upload the file using the path
  myfile = genai.upload_file(image_path)

  model = genai.GenerativeModel("gemini-1.5-flash")
  result = model.generate_content(
      [myfile,
       "\n\n",
       "You are a highly skilled OCR system designed to extract information from product packaging. \n\n"
       "**Your task:**\n"
       "- Read the text provided in the image.\n"
       "- Identify and extract the following key details:\n"
       "    - **Manufacturer:** The name of the company that produced the product.\n"
       "    - **Product Name:** The name of the product itself.\n"
       "    - **Ingredients:** A list of ingredients, if available.\n"
       "    - **Manufacturing Date:** The date when the product was manufactured (format: YYYY-MM-DD).\n"
       "    - **Expiry Date:** The date when the product expires (format: YYYY-MM-DD).\n"
       "    - **Net Weight:** The weight of the product (include units, e.g., 100g, 500ml).\n"
       "    - **Barcode:** The barcode number, if visible.\n"
       "    - **Other Relevant Details:** Any other crucial information you find on the packaging.\n\n"
       "**Output Format:**\n\n"
       "Provide your answer in a structured format, using JSON:\n"
       "```json\n"
       "{ \n"
       "  \"Manufacturer\": \"Manufacturer Name\", \n"
       "  \"Product Name\": \"Product Name\", \n"
       "  \"Ingredients\": [\"Ingredient 1\", \"Ingredient 2\", ...], \n"
       "  \"Manufacturing Date\": \"YYYY-MM-DD\", \n"
       "  \"Expiry Date\": \"YYYY-MM-DD\", \n"
       "  \"Net Weight\": \"Quantity with Units\", \n"
       "  \"Barcode\": \"Barcode Number\", \n"
       "  \"Other Relevant Details\": \"Any other information found\" \n"
       "}\n"
       "```\n\n"
       "***Example:***\n"
       "Image: [Image of a product packaging]\n\n"
       "Output:\n"
       "{ \n"
       "  \"Manufacturer\": \"Acme Foods\", \n"
       "  \"Product Name\": \"Organic Tomato Sauce\", \n"
       "  \"Ingredients\": [\"Tomatoes\", \"Salt\", \"Sugar\", \"Basil\"], \n"
       "  \"Manufacturing Date\": \"2024-03-15\", \n"
       "  \"Expiry Date\": \"2025-03-14\", \n"
       "  \"Net Weight\": \"500g\", \n"
       "  \"Barcode\": \"8765432109876\", \n"
       "  \"Other Relevant Details\": \"Contains gluten. Store in a cool, dry place.\" \n"
       "}\n"
       ]
  )
  return result.parts[0].text

# Upload multiple images
uploaded = files.upload()

# Process each image
results = []
for image_filename in uploaded:
  # Access the uploaded image
  image = Image.open(io.BytesIO(uploaded[image_filename]))  # Open the image

  # Save the image to a temporary file
  temp_image_path = f"/content/{image_filename}"  # Use the original filename
  image.save(temp_image_path)  # Save the image

  # Start timing
  start_time = time.time()

  # Process the image and store results
  result = process_image(temp_image_path)
  results.append(result)

  # Stop timing and calculate the duration
  end_time = time.time()
  duration = end_time - start_time

  print(f"Image: {image_filename}")
  print(f"Time taken: {duration:.2f} seconds")  # Print the duration

  # Delete the temporary image file
  os.remove(temp_image_path)

# Print the results
print("Results:")
for i, result in enumerate(results):
  print(f"Result {i + 1}:")
  print(result)